In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import re

Import de données

In [2]:
CAC40_txt = """Accor SA (AC.PA)
L'Air Liquide S.A. (AI.PA)
Airbus SE (AIR.PA)
ArcelorMittal S.A. (MT.AS)
AXA SA (CS.PA)
BNP Paribas SA (BNP.PA)
Bouygues SA (EN.PA)
Bureau Veritas SA (BVI.PA)
Capgemini SE (CAP.PA)
Carrefour SA (CA.PA)
Crédit Agricole S.A. (ACA.PA)
Danone S.A. (BN.PA)
Dassault Systèmes SE (DSY.PA)
Edenred SE (EDEN.PA)
Engie SA (ENGI.PA)
EssilorLuxottica Société anonyme (EL.PA)
Eurofins Scientific SE (ERF.PA)
Hermès International (RMS.PA)
Kering SA (KER.PA)
Legrand SA (LR.PA)
L'Oréal S.A. (OR.PA)
LVMH (MC.PA)
Michelin (ML.PA)
Orange S.A. (ORA.PA)
Pernod Ricard SA (RI.PA)
Publicis Groupe S.A. (PUB.PA)
Renault SA (RNO.PA)
Safran SA (SAF.PA)
Compagnie de Saint-Gobain S.A. (SGO.PA)
Sanofi (SAN.PA)
Schneider Electric S.E. (SU.PA)
Société Générale (GLE.PA)
Stellantis N.V. (STLAP.PA)
STMicroelectronics N.V. (STMPA.PA)
Teleperformance SE (TEP.PA)
Thales S.A. (HO.PA)
TotalEnergies SE (TTE.PA)
Unibail-Rodamco-Westfield SE (URW.PA)
Veolia Environnement SA (VIE.PA)
Vinci SA (DG.PA)"""

tickers_CAC40 = re.findall(r'\(([^)]+)\)', CAC40_txt)

In [3]:
histo_CAC40 = yf.download(tickers = tickers_CAC40, start = '2015-01-01',end = '2025-02-01')["Close"]
histo_CAC40_maille1h =  yf.download(tickers = tickers_CAC40, start = '2024-01-01',end = '2025-02-01', interval='1h')["Close"]

[*********************100%***********************]  40 of 40 completed
[*********************100%***********************]  40 of 40 completed


In [4]:
print(histo_CAC40.shape)
histo_CAC40["TEP.PA"].head(10)

(2583, 40)


Date
2015-01-02    48.126030
2015-01-05    47.900623
2015-01-06    47.124268
2015-01-07    47.216095
2015-01-08    48.142715
2015-01-09    47.666882
2015-01-12    49.920837
2015-01-13    50.104492
2015-01-14    49.879093
2015-01-15    50.588669
Name: TEP.PA, dtype: float64

Data cleaning

In [5]:
nan_par_colonne = {col: histo_CAC40.index[histo_CAC40[col].isna()].tolist() for col in histo_CAC40.columns}
for col in histo_CAC40.columns:
    taille_liste = len(nan_par_colonne[col])
    print(col, len(nan_par_colonne[col])) if taille_liste > 0 else next
    #print(col, len(nan_par_colonne[col]), nan_par_colonne[col]) if taille_liste > 0 else next


MT.AS 1
URW.PA 2122


In [6]:
histo_CAC40_sans_MT_URW = histo_CAC40.drop(columns=["URW.PA", "MT.AS"])
print(int(histo_CAC40_sans_MT_URW.isna().sum().sum()))
print(histo_CAC40_sans_MT_URW.shape)

0
(2583, 38)


Calcul des rendements, volatilités et corrélations

In [7]:
data = histo_CAC40_sans_MT_URW
# Calcul du vecteur de rendement moyen annuel
mu_annuel = np.log((data.iloc[-1, :]/data.iloc[0,:])) / (data.index[-1] - data.index[0]).days * 365.25

print("Rendements annuels :")
print(mu_annuel.head())

Rendements annuels :
Ticker
AC.PA     0.043649
ACA.PA    0.088683
AI.PA     0.112176
AIR.PA    0.150802
BN.PA     0.052707
dtype: float64


In [8]:
delta_année = data.index.to_series().diff().dt.days.iloc[1:] / 365.25
deltat_r = pd.DataFrame(delta_année.values[:, None] * mu_annuel.values, columns=mu_annuel.index, index=delta_année.index)
nb_jour_pan = data.shape[0] / (data.index[-1] - data.index[0]).days * 365.25
Volatilite_annuel = np.sqrt((np.log((data/data.shift(1)).iloc[1:,:]) - deltat_r).var() * nb_jour_pan)
mat_correlation = (np.log((data/data.shift(1)).iloc[1:,:]) - deltat_r).corr()

print("\nVolatilités :")
print(Volatilite_annuel.head())


Volatilités :
Ticker
AC.PA     0.319046
ACA.PA    0.309398
AI.PA     0.206439
AIR.PA    0.351973
BN.PA     0.194999
dtype: float64


In [9]:
print("\nMatrice de correlation :")
print(mat_correlation.iloc[ :7, :7])


Matrice de correlation :
Ticker     AC.PA    ACA.PA     AI.PA    AIR.PA     BN.PA    BNP.PA    BVI.PA
Ticker                                                                      
AC.PA   1.000000  0.585502  0.428098  0.634105  0.354309  0.602450  0.451828
ACA.PA  0.585502  1.000000  0.480506  0.558192  0.339130  0.853126  0.470121
AI.PA   0.428098  0.480506  1.000000  0.484603  0.499491  0.499644  0.519341
AIR.PA  0.634105  0.558192  0.484603  1.000000  0.373528  0.591485  0.465776
BN.PA   0.354309  0.339130  0.499491  0.373528  1.000000  0.367019  0.406833
BNP.PA  0.602450  0.853126  0.499644  0.591485  0.367019  1.000000  0.471934
BVI.PA  0.451828  0.470121  0.519341  0.465776  0.406833  0.471934  1.000000
